In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt
import cv2
import os
import dlib
import cv2
import imutils
from imutils.face_utils import FaceAligner
import os
import numpy as np
import pandas as pd
import keras
import shutil
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, img_to_array

from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Dense, Activation, Dropout, Flatten, Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Activation
from keras.layers import Conv2D, AveragePooling2D
from keras.models import Model, Sequential

os.environ["CUDA_VISIBLE_DEVICES"]="6"
tqdm.pandas()

# fotograflarin getirilecegi boyut
target_size = (224, 224)

# yuz tespiti icin detektor
face_detector = dlib.get_frontal_face_detector()

In [ ]:
# fotografta yuz var mi kontrolu
def check_detection(img):
    exact_image = False
    if type(img).__module__ == np.__name__:
        exact_image = True

    base64_img = False
    if len(img) > 11 and img[0:11] == "data:image/":
        base64_img = True

    elif not exact_image:  # image path passed as input

        if not os.path.isfile(img):
            raise ValueError("Confirm that ", img, " exists")

        img = cv2.imread(img)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rects, scores, idx = face_detector.run(gray,1,-1)
            
    if len(rects) > 0:
        return True
    else:
        return False
    
# yuz tespit eden fonksyion
def detect_face(img):
    exact_image = False
    if type(img).__module__ == np.__name__:
        exact_image = True

    base64_img = False
    if len(img) > 11 and img[0:11] == "data:image/":
        base64_img = True

    elif not exact_image:  # image path passed as input
        if not os.path.isfile(img):
            raise ValueError("Confirm that ", img, " exists")

        img = cv2.imread(img)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # rects: yuz tespiti yapilan cerceve
    # scores: yuz tespiti skorlari
    # best_score_index: en yuksek skora sahip yuzun indisi
    rects, scores, idx = face_detector.run(gray,1,-1)
    best_score_index = np.argmax(scores)
    faces = []
    # tespit edilen yuzler fotograftan kirpiliyor 
    for rect in rects:
        detected_face = img[max(0, rect.top()): min(rect.bottom(), img.shape[0]),
                    max(0, rect.left()): min(rect.right(), img.shape[1])]
        
        # fotograflar 224, 224 hale getiriliyor
        detected_face = cv2.resize(detected_face, (target_size[0], target_size[1]))
        faces.append(detected_face)

    return faces[best_score_index]

In [ ]:
# veri okundu
train_df = pd.read_csv("fairface/fairface_label_train.csv")
test_df = pd.read_csv("fairface/fairface_label_val.csv")

In [ ]:
print("train set: ",train_df.shape)
print("test set: ",test_df.shape)

In [ ]:
train_df = train_df[['file', 'race']]
test_df = test_df[['file', 'race']]

In [ ]:
train_df['file'] = 'fairface/'+train_df['file']
test_df['file'] = 'fairface/'+test_df['file']

In [ ]:
train_df.head()

In [ ]:
# veri setindeki ırklara gore veri dagilimi
100*train_df.groupby(['race']).count()[['file']]/train_df.groupby(['race']).count()[['file']].sum()

In [ ]:
# East ve Southeast Asian, Asian olarak birlestiriliyor.
idx = train_df[(train_df['race'] == 'East Asian') | (train_df['race'] == 'Southeast Asian')].index
train_df.loc[idx, 'race'] = 'Asian'

idx = test_df[(test_df['race'] == 'East Asian') | (test_df['race'] == 'Southeast Asian')].index
test_df.loc[idx, 'race'] = 'Asian'

In [ ]:
100*train_df.groupby(['race']).count()[['file']]/train_df.groupby(['race']).count()[['file']].sum()

In [ ]:
train_df.head()

In [ ]:
# yuz tespiti kontrolu
train_df['detection'] = train_df['file'].progress_apply(check_detection)
test_df['detection'] = test_df['file'].progress_apply(check_detection)

In [ ]:
test_df['detection'].value_counts()

In [ ]:
# yuz tespit edilemeyen fotograflar cikartiliyor
train_df = train_df[train_df.detection == True]
test_df = test_df[test_df.detection == True]

In [ ]:
# yuz tespiti
train_df['pixels'] = train_df['file'].progress_apply(detect_face)
test_df['pixels'] = test_df['file'].progress_apply(detect_face)

In [ ]:
train_features = []
test_features = []

for i in range(0, train_df.shape[0]):
    train_features.append(train_df['pixels'].values[i])

for i in range(0, test_df.shape[0]):
    test_features.append(test_df['pixels'].values[i])

In [ ]:
# verilerin egitimde kullanilabilmesi icin uygun formata getiriliyor
train_features = np.array(train_features)
test_features = np.array(test_features)

train_features = train_features / 255
test_features = test_features / 255

In [ ]:
train_label = train_df[['race']]
test_label = test_df[['race']]

In [ ]:
races = train_df['race'].unique()

In [ ]:
# labellar categorical hale getiriliyor
for j in range(len(races)):
    current_race = races[j]
    print("replacing ",current_race," to ", j+1)
    train_label['race'] = train_label['race'].replace(current_race, str(j+1))
    test_label['race'] = test_label['race'].replace(current_race, str(j+1))

train_label = train_label.astype({'race': 'int32'})
test_label = test_label.astype({'race': 'int32'})

In [ ]:
train_label.head()

In [ ]:
train_target = pd.get_dummies(train_label['race'], prefix='race')
test_target = pd.get_dummies(test_label['race'], prefix='race')

In [ ]:
train_target.head()

## Validation set

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(train_features, train_target.values
                                        , test_size=0.12, random_state=17)

# Model

In [ ]:
import keras
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Dense, Activation, Dropout, Flatten, Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Activation
from keras.layers import Conv2D, AveragePooling2D
from keras.models import Model, Sequential

In [ ]:
#VGG-Face model
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [ ]:
# vgg-face agirliklari yuklendi
model.load_weights('vgg_face_weights.h5')

In [ ]:
num_of_classes = 6

In [ ]:
# son 7 katman egitime acildi, ek katmanlar eklendi
for layer in model.layers[:-7]:
    layer.trainable = False

base_model_output = Sequential()
base_model_output = Convolution2D(num_of_classes, (1, 1), name='predictions')(model.layers[-4].output)
base_model_output = Flatten()(base_model_output)
base_model_output = Activation('softmax')(base_model_output)

race_model = Model(inputs=model.input, outputs=base_model_output)

In [ ]:
race_model.compile(loss='categorical_crossentropy'
                  , optimizer=keras.optimizers.Adam()
                  , metrics=['accuracy']
                 )

# Egitim

In [ ]:
# /models klasoru olmasi gereklidir. Her tur sonunda en iyi model kontrolu yapilarak kaydedilecektir. 
checkpointer = ModelCheckpoint(
    filepath='race_model.hdf5'
    , monitor = "val_loss"
    , verbose=1
    , save_best_only=True
    , mode = 'auto'
)

In [ ]:
# patience early stopping icin gerekli,
# val_loss'un azalmasinin kac epoch beklenecegini belirtir. Azalmazsa egitimi durdurur.
patience = 3
epochs = 10

early_stop = EarlyStopping(monitor='val_loss', patience=patience) 

score = race_model.fit(
    train_x, train_y
    , epochs=epochs
    , validation_data=(val_x, val_y)
    , callbacks=[checkpointer, early_stop]
)

In [ ]:
# en iyi model yukleniyor
from keras.models import load_model
race_model = load_model("race_model.hdf5")


In [ ]:
test_perf = race_model.evaluate(test_features, test_target.values, verbose=1)
print(test_perf)

In [ ]:
validation_perf = race_model.evaluate(val_x, val_y, verbose=1)
print(validation_perf)

In [ ]:
predictions = race_model.predict(test_features)

In [ ]:
prediction_classes = []; actual_classes = []

for i in range(0, predictions.shape[0]):
    prediction = np.argmax(predictions[i])
    prediction_classes.append(races[prediction])
    actual = np.argmax(test_target.values[i])
    actual_classes.append(races[actual])

# Confusion Matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sn

In [ ]:
cm = confusion_matrix(actual_classes, prediction_classes)
cm

In [ ]:
df_cm = pd.DataFrame(cm, index=races, columns=races)
sn.heatmap(df_cm, annot=True,annot_kws={"size": 10})